In [4]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName('returning users').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/20 07:54:54 WARN Utils: Your hostname, Pulastyas-Mac-mini.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.133 instead (on interface en1)
26/01/20 07:54:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/20 07:54:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [25]:
trans = spark.read.options(header=True, inferSchema=True)\
    .csv('data/input/amazon_transactions.csv')

trans.show(truncate=False)

+---+-------+-------+----------+-------+
|id |user_id|item   |created_at|revenue|
+---+-------+-------+----------+-------+
|1  |109    |milk   |2020-03-03|123    |
|2  |139    |biscuit|2020-03-18|421    |
|3  |120    |milk   |2020-03-18|176    |
|4  |108    |banana |2020-03-18|862    |
|5  |130    |milk   |2020-03-28|333    |
|6  |103    |bread  |2020-03-29|862    |
|7  |122    |banana |2020-03-07|952    |
|8  |125    |bread  |2020-03-13|317    |
|9  |139    |bread  |2020-03-30|929    |
|10 |141    |banana |2020-03-17|812    |
|11 |116    |bread  |2020-03-31|226    |
|12 |128    |bread  |2020-03-04|112    |
|13 |146    |biscuit|2020-03-04|362    |
|14 |119    |banana |2020-03-28|127    |
|15 |142    |bread  |2020-03-09|503    |
|16 |122    |bread  |2020-03-06|593    |
|17 |128    |biscuit|2020-03-24|160    |
|18 |112    |banana |2020-03-24|262    |
|19 |149    |banana |2020-03-29|382    |
|20 |100    |banana |2020-03-18|599    |
+---+-------+-------+----------+-------+
only showing top

In [38]:
trans1 = trans.select(F.col('user_id'), F.col('created_at'))\
    .withColumn('first_purc_at', F.first(F.col('created_at')).over(Window.partitionBy('user_id').orderBy('created_at')))\
    .filter(F.col('created_at') != F.col('first_purc_at'))\
    .withColumn('days_since_first_purc', F.datediff(F.col('created_at'), F.col('first_purc_at')))\
    .filter(F.col('days_since_first_purc') <= 7)\
    .select('user_id')\
    .dropDuplicates()


trans1.show()

+-------+
|user_id|
+-------+
|    100|
|    103|
|    105|
|    109|
|    111|
|    114|
|    117|
|    122|
|    130|
|    131|
|    133|
|    141|
|    143|
+-------+



26/01/20 13:58:18 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 989134 ms exceeds timeout 120000 ms
26/01/20 13:58:18 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/20 13:58:26 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$